<a href="https://colab.research.google.com/github/K0EKJE/CNN_compression_with_Tensor_Decomposition/blob/main/CNN_compression_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Compression With Tensor Decomposition Demo

First install `tensorly` and clone the repo to access pretrained model weights and scripts.

In [1]:
pip install tensorly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 5.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/K0EKJE/CNN_compression_with_Tensor_Decomposition.git

Cloning into 'CNN_compression_with_Tensor_Decomposition'...
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 303 (delta 77), reused 141 (delta 69), pack-reused 150
Receiving objects: 100% (303/303), 1016.19 MiB | 35.62 MiB/s, done.
Resolving deltas: 100% (129/129), done.


## CP decomposition

Run the follwoing to start decomposition on the second convolutional layer with CP decomposition. It will download the dataset first and print the decomposed model  structure. Check the index for convolutional layers. A `decomposed_model` file will be automatically generated in the directory.

**Note:** change `layer_to_decomp_cp` in config file to choose layer for decomposition. As for CP decomposition, it requires large RAM so try to test with only the frist first two convolutiaon layers.

In [3]:
!python /content/CNN_compression_with_Tensor_Decomposition/main.py --decompose

==> Preparing data..
100% 170498071/170498071 [00:05<00:00, 30635929.26it/s]
Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Decomposing layer 3: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
CP rank = 21
Decomposition of layer 3 Completed. Ratio = 0.7589842


==> Building decomposed model..
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): Conv2d(64, 21, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(21, 21, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=21, bias=False)
      (2): Conv2d(21, 21, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=21, bias=False)
      (3): Conv2d(21, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

We have successfully used a rank-21 CP decomposition to decomposed the second convolutional layer of the model, and we can see that the accuracy right after decomposition is 84.240%(it may vary for each run). For reference, the base accuracy is 93.11%. After three epochs, the accuracy recovers to 91.950%.

Now we can finetune for 3 epochs, and we will see a quick recovery of accuracy after the first epoch. The accruacy might not keep reducing as we are using a relatively large learning rate. Usually we will keep a small learning rate to avoid over-training.

In [4]:
!python /content/CNN_compression_with_Tensor_Decomposition/main.py --run_model --fine_tune --lr=0.001

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): Conv2d(64, 21, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(21, 21, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=21, bias=False)
      (2): Conv2d(21, 21, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=21, bias=False)
      (3): Conv2d(21, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, moment

## Partial Tucker Decomposition

Add a `--tucker` parser to use the Tucker decomposition.The rank selection method can be chosen from VBMF and SVD by changing the `tucker_rank_selection_method` setting in the config file, and similarly `layer_to_decomp_tucker` will control layers to decompose.

By default, it will apply SVT to all the layers.

In [5]:
!python /content/CNN_compression_with_Tensor_Decomposition/main.py --decompose --tucker

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Decomposing layer 3: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
SVD Estimated ranks:  [15, 14]
Number of params before: 36928 || after: 3839
Decomposition of layer 3 Completed. Ratio = 0.78914803

Decomposing layer 7: Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
SVD Estimated ranks:  [27, 22]
Number of params before: 73856 || after: 10387
Decomposition of layer 7 Completed. Ratio = 0.7905847

Decomposing layer 10: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
SVD Estimated ranks:  [47, 41]
Number of params before: 147584 || after: 28823
Decomposition of layer 10 Completed. Ratio = 0.7721972

Decomposing layer 14: Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
SVD Estimated ranks:  [79, 66]
Number of params before: 295168 || after: 75999
Decomposition of layer 14 Completed. Ratio = 0.7958527

Decomposing 

We can see that the decomposed model achieves 20022592 ==> 1122657(X17.84) compression, but it currenly loses predictive ability. We will see the accuracy recovering quickly with merely one epoch of training.

Let's also fine-tune for a few epochs. Change `fine_tune_epochs` to vary the number of epochs. The fine-tuned model will be automatically saved as `fine_tuned_model` after training complete..

In [6]:
!python /content/CNN_compression_with_Tensor_Decomposition/main.py --run_model --fine_tune --lr=0.001

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): Conv2d(64, 14, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(14, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): Conv2d(15, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Sequential(
      (0): Conv2d(64, 22, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(22, 27, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): Conv2d(27, 128, kernel_size=(1, 1), 